In [5]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep

import pymysql

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options = options)

url = "https://m.stock.naver.com/index.html#/domestic/capitalization/KOSPI"
driver.get(url)

In [6]:
sleep(5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
sleep(1)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
sleep(1)
more = driver.find_element_by_xpath('//*[@id="content"]/div[4]/div[2]/div[2]/div[2]/a')
more.click()
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
driver.implicitly_wait(1)
more.click()

In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

stock_code = soup.findAll('span',{'class':'VTableComm_code__3xLPv'})

In [8]:
stock_urls = []
for code in stock_code:
    stock_urls.append(f'https://m.stock.naver.com/index.html#/domestic/stock/{code.text}/total')

In [25]:
import pandas as pd
from time import sleep
SQL = ''
data1=[]
data2=[]
for url in stock_urls:
    
    driver.get(url)
    sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    stock_id = soup.find('span',{'class':'VGraphMain_code__6CPM3'}).text[:-3]
    name = soup.find('span',{'class':'VGraphMain_name__21QMQ'}).text
    elements_values = soup.findAll('span',{'class':'VStockInfo_value__3D1se'})

    elements = []
    elements.append(int(stock_id))
    elements.append(name)
    for element in elements_values[:12]:
        elements.append(element.text)
        
    updownrate = soup.findAll('span',{'class':'VGap_gap__290GV'})[1].text
    elements.append(updownrate)
    data1.append(elements)
    
    headlines = soup.findAll('p',{'class':'VNewsList_title__3ORgG'})
    short = soup.findAll('p',{'class':'VNewsList_text__2VCjU'})
    news = []
    for idx, headline in enumerate(headlines):
        data2.append([int(stock_id),headline.text,short[idx].text])

        
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='stock', charset='utf8')

cursor = db.cursor()
db.autocommit(True)

SQL = 'INSERT INTO stock VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
cursor.executemany(SQL,data1)

db.close()


db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='stock', charset='utf8')

cursor = db.cursor()
db.autocommit(True)

SQL = 'INSERT INTO news VALUES(%s,%s,%s)'
cursor.executemany(SQL,data2)

db.close()
driver.close()

InvalidSessionIdException: Message: invalid session id


In [22]:
import pymysql
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='1234', db='stock', charset='utf8')

cursor = db.cursor()
db.commit()
db.close()

In [23]:
SQL = """CREATE TABLE `stock`.`stock` (
  `id` INT NOT NULL,
  `name` VARCHAR(45) NULL,
  `yp` VARCHAR(45) NULL,
  `pp` VARCHAR(45) NULL,
  `hp` VARCHAR(45) NULL,
  `lp` VARCHAR(45) NULL,
  `ta` VARCHAR(45) NULL,
  `cp` VARCHAR(45) NULL,
  `mc` VARCHAR(45) NULL,
  `fc` VARCHAR(45) NULL,
  `52h` VARCHAR(45) NULL,
  `52l` VARCHAR(45) NULL,
  `per` VARCHAR(45) NULL,
  `eps` VARCHAR(45) NULL,
  `r` VARCHAR(45) NULL,
  PRIMARY KEY (`id`));
"""

cursor.execute(SQL)
    
db.commit()

db.close()

InterfaceError: (0, '')

In [20]:
SQL = """CREATE TABLE `stock`.`news` (
  `id` INT NOT NULL,
  `headline` VARCHAR(100) NULL,
  `short` VARCHAR(500) NULL);
"""
cursor = db.cursor()
cursor.execute(SQL)
    
db.commit()

db.close()

InterfaceError: (0, '')